Leung Wai Liu <br>
JPMorgan AI Research<br>
August 8, 2022 <br>
Twitter Relabelling TRAINING - VERSION 2

In [1]:
import pandas as pd
import numpy as np
import wordninja
import spacy
from collections import Counter
from labels_to_tweebank import labels_to_tweebank

In [2]:
nlp = spacy.load('en_core_web_trf')

In [3]:
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [4]:
combined_df = pd.read_csv('logs/combined_train_labels_FINAL.tsv', sep='\t')
combined_df

,Word,Start Word Index,End Word Index,POS Label,NER Label,Tweet,Tweet POS Labels,Tweet NER Labels
0,poutine,11,11,['PROPN'],['O'],rt @user1787 : i asked siri to remind me about...,"['X', 'X', 'PUNCT', 'PRON', 'VERB', 'PROPN', '...","['O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', '..."
1,friday,7,7,['PROPN'],['O'],rt @user912 : you know it 's friday when the p...,"['X', 'X', 'PUNCT', 'PRON', 'VERB', 'PRON', 'A...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,@user444,17,17,['PROPN'],['O'],mikel obi & his girl olga diyachenko spend hol...,"['PROPN', 'PROPN', 'CCONJ', 'PRON', 'NOUN', 'P...","['B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'I-..."
3,@user1598,1,1,['PROPN'],['O'],""" @user1598 : super junior ’s siwon transforms...","['PUNCT', 'PROPN', 'PUNCT', 'PROPN', 'PROPN', ...","['O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'B-PER'..."
4,gospel,7,7,['PROPN'],['O'],""" do n't move on from the gospel , move into t...","['PUNCT', 'AUX', 'PART', 'VERB', 'ADP', 'ADP',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...,...,...,...,...
966,@user70,3,3,['PROPN'],['O'],rt @user2041 : @user70 happy birthday man ! le...,"['X', 'X', 'PUNCT', 'PROPN', 'ADJ', 'NOUN', 'N...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
967,@user660,5,5,['PROPN'],['O'],rt @user1157 : happy birthday @user660 ! url1209,"['X', 'X', 'PUNCT', 'ADJ', 'NOUN', 'PROPN', 'P...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"
968,@user2046,0,0,['PROPN'],['O'],@user2046 further you do the swedish social se...,"['PROPN', 'ADV', 'PRON', 'VERB', 'DET', 'ADJ',...","['O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', ..."
969,@user2242,20,20,['PROPN'],['O'],@user2046 further you do the swedish social se...,"['PROPN', 'ADV', 'PRON', 'VERB', 'DET', 'ADJ',...","['O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', ..."


In [5]:
# Changing all @user to be B-PER for now
new_label = combined_df['NER Label'].tolist()
new_label_before_convert = list(new_label)
new_pos = combined_df['POS Label'].tolist()
new_pos_before_convert = list(new_pos)
new_stand_out_tweet = combined_df['Tweet'].tolist()

for index, row in combined_df.iterrows():
    
    # retrieving the the information of a particular row
    particular_word = row['Word']
    particular_start_index = row['Start Word Index']
    particular_end_index = row['End Word Index'] + 1
    num_words_particular = particular_end_index - particular_start_index
    original_sentence = row['Tweet']

    # splitting the original sentence
    original_sentence_split = original_sentence.split(' ')

    # putting the brackets in the original sentence to signify word phrase:
    word_phrase_stand_out = list(original_sentence_split)
    word_phrase_stand_out[particular_start_index] = "<[" + word_phrase_stand_out[particular_start_index]
    word_phrase_stand_out[particular_end_index - 1] = word_phrase_stand_out[particular_start_index] + "]>"
    word_phrase_stand_out_string = ' '.join(word_phrase_stand_out)
    new_stand_out_tweet[index] = word_phrase_stand_out_string
    
    
    print('-' * 50)
    print('Original Particular word')
    print(particular_word)
    print("Original Sentence Split")     
    print(original_sentence_split)
    print("Original Start Index")
    print(particular_start_index)

    # setting up the initial parameters  
    char_index = 0
    iterate_word_index = 0
    is_hashtag = False
    is_user = False
    encode = ''
    encode_len = 0

    print("FINDING ENTITY OF WORD")

    pre_processed_sentence_split = list(original_sentence_split)
    pre_processed_word_start_index = particular_start_index
    pre_processed_word_end_index = particular_end_index
    pre_processed_char_start_index = 0
    pre_processed_char_start_index_mushed = 0
    pre_processed_sentence_split_string = ""

    # NO SPLITTING, FEEDING @USER TAG IN IT

    word_index = 0 
    while word_index in range(len(pre_processed_sentence_split)):
        word_in_sentence = pre_processed_sentence_split[word_index]
        check_for_hashtag = word_in_sentence[0:1]

        # if hashtag, remove the hashtag
        if check_for_hashtag == '#':
            word_in_sentence = word_in_sentence[1:]
            pre_processed_sentence_split[word_index] = word_in_sentence
        
        to_add_word_string_length = len(word_in_sentence)

        word_index += 1
    
    

    pre_processed_sentence_split_string = ' '.join(pre_processed_sentence_split)

    # Getting the specific character of the post pre-processed sentence
    part_of_sentence_before_particular_word = pre_processed_sentence_split[:pre_processed_word_start_index]
    part_of_sentence_before_particular_word_string = ' '.join(part_of_sentence_before_particular_word)

    pre_processed_char_start_index = (len(part_of_sentence_before_particular_word_string)) + 1

    # Without getting the string without spaces
    part_of_sentence_before_particular_word_string_mushed = ''.join(part_of_sentence_before_particular_word)
    pre_processed_char_start_index_mushed = (len(part_of_sentence_before_particular_word_string_mushed))

    # processing particular word phrase after pre-processing
    particular_word_after_processing = pre_processed_sentence_split[pre_processed_word_start_index: pre_processed_word_end_index]
    particular_word_string_after_processing = ''.join(particular_word_after_processing)
    len_particular_word_string_after_processing_mushed = len(particular_word_string_after_processing)

    # encoding into spacy document to get the NER entity of each word
    sentence_doc = nlp(pre_processed_sentence_split_string) # splitted word phrase
    word_entities = [(e.text, e.ent_iob_, e.ent_type_, e.pos_) for e in sentence_doc]

    # check if the length of the entities that fed in matches with the len of entities that came out
    
    print("FOR NON COMBINED WORDS")

    print("After processing")
    print(pre_processed_sentence_split)
    print("After processing joint sentence")
    print(pre_processed_sentence_split_string)
    print("Pre processed word start index")
    print(pre_processed_word_start_index)
    print("Pre processed char start index")
    print(pre_processed_char_start_index)
    print("Pre processed split string")
    print(pre_processed_sentence_split_string)
    print("Part of sentence before particular word")
    print(part_of_sentence_before_particular_word)
    print("Part of sentence before particular word string")
    print(part_of_sentence_before_particular_word_string)
    print("Part of sentence before particular word string mushed")
    print(part_of_sentence_before_particular_word_string_mushed)
    print("FINAL: Pre Processed char start index mushed")
    print(pre_processed_char_start_index_mushed)    
    print("LEN OF WORD PHRASE AFTER MUSHED")
    print(len_particular_word_string_after_processing_mushed)
    
    print('Word Entities result')
    print(word_entities)

    # retrieving the entities to verify:

    # NON COMBINED SENTENCE 
    entities_index_non_combined = 0
    entities_char_non_combined = 0

    word_entities_to_verify_non_combined = []
    
    for entity in word_entities:
        if entities_char_non_combined < pre_processed_char_start_index_mushed:
            particular_entity_word = entity[0]
            particular_entity_word_length = len(particular_entity_word)
            entities_index_non_combined += 1
            entities_char_non_combined += particular_entity_word_length
        
    
    while entities_char_non_combined < (pre_processed_char_start_index_mushed + len_particular_word_string_after_processing_mushed):
        entity_to_append_non_combined = word_entities[entities_index_non_combined]
        word_entities_to_verify_non_combined.append(entity_to_append_non_combined)
        
        entities_char_non_combined += len(entity_to_append_non_combined[0])
        entities_index_non_combined += 1
    
    print("PRINTING NON COMBINED")
    print(word_entities_to_verify_non_combined)

    # retrieving all the labels and pos tags from each phrase (if any)
    label = []
    pos = []
    
    # otherwise append all of the NER tags from the split word entity (if there are)
    for numents in range(len(word_entities_to_verify_non_combined)):
        if word_entities_to_verify_non_combined[numents][2] != '':
            label.append(word_entities_to_verify_non_combined[numents][2])
        pos.append(word_entities_to_verify_non_combined[numents][3])
    
    print(label)
    print(pos)

    if label != []:
        new_label_before_convert[index] = list(label)
    else:
        new_label_before_convert[index] = []

    new_pos_before_convert[index] = list(pos)
    
    # Filtering - if the label list is not empty: 
    if label != []:
        label = [labels_to_tweebank[label_item] for label_item in label]
        
        if len(label) == num_words_particular:
            for num_word in range(len(label)):
                if num_word == 0:
                    label[num_word] = 'B-' + label[num_word]
                else:
                    label[num_word] = 'I-' + label[num_word]
        elif len(label) > num_words_particular:
            for num_word in range(num_words_particular):
                if num_word == 0:
                    label[num_word] = 'B-' + label[num_word]
                else:
                    label[num_word] = 'I-' + label[num_word]
        elif len(label) < num_words_particular:
            label_counter = Counter(label)
            most_common_label = label_counter.most_common(1)[0][0]
            label = []
            for num_word in range(num_words_particular):
                if num_word == 0:
                    label.append('B-' + most_common_label)
                else:
                    label.append('I-' + most_common_label)
        
        new_label[index] = label
        new_pos[index] = pos

    else:
        if '@user' in particular_word:
            new_label[index] = ['B-PER']
            new_pos[index] = pos
        elif 'PROPN' in pos:
            label = []
            for num_word in range(num_words_particular):
                if num_word == 0:
                    label.append('B-' + 'MISC')
                else:
                    label.append('I-' + 'MISC')
            new_label[index] = label
            new_pos[index] = pos
    
 


--------------------------------------------------
Original Particular word
poutine
Original Sentence Split
['rt', '@user1787', ':', 'i', 'asked', 'siri', 'to', 'remind', 'me', 'about', 'my', 'poutine', 'in', 'the', 'office', 'fridge', 'but', 'she', 'misinterpreted', 'and', 'probably', 'alerted', 'the', 'fbi', '.', 'url222', '…']
Original Start Index
11
FINDING ENTITY OF WORD
FOR NON COMBINED WORDS
After processing
['rt', '@user1787', ':', 'i', 'asked', 'siri', 'to', 'remind', 'me', 'about', 'my', 'poutine', 'in', 'the', 'office', 'fridge', 'but', 'she', 'misinterpreted', 'and', 'probably', 'alerted', 'the', 'fbi', '.', 'url222', '…']
After processing joint sentence
rt @user1787 : i asked siri to remind me about my poutine in the office fridge but she misinterpreted and probably alerted the fbi . url222 …
Pre processed word start index
11
Pre processed char start index
50
Pre processed split string
rt @user1787 : i asked siri to remind me about my poutine in the office fridge but she m

In [6]:
new_df = combined_df
new_df['Spacy NER Label'] = new_label_before_convert
new_df['Spacy POS Label'] = new_pos_before_convert

new_df['Final NER Label'] = new_label
new_df['Final POS Label'] = new_pos

new_df['Candidate Label'] = new_stand_out_tweet

new_df = new_df.reindex(columns = ['Word', 'POS Label', 'Spacy POS Label', 'Final POS Label', 'NER Label', 'Spacy NER Label', 'Final NER Label', 'Tweet', 'Candidate Label'])
new_df

,Word,POS Label,Spacy POS Label,Final POS Label,NER Label,Spacy NER Label,Final NER Label,Tweet,Candidate Label
0,poutine,['PROPN'],[NOUN],['PROPN'],['O'],[],['O'],rt @user1787 : i asked siri to remind me about...,rt @user1787 : i asked siri to remind me about...
1,friday,['PROPN'],[PROPN],[PROPN],['O'],[],[B-MISC],rt @user912 : you know it 's friday when the p...,rt @user912 : you know it 's <[friday]> when t...
2,@user444,['PROPN'],[X],[X],['O'],[],[B-PER],mikel obi & his girl olga diyachenko spend hol...,mikel obi & his girl olga diyachenko spend hol...
3,@user1598,['PROPN'],[PROPN],[PROPN],['O'],[],[B-PER],""" @user1598 : super junior ’s siwon transforms...",""" <[@user1598]> : super junior ’s siwon transf..."
4,gospel,['PROPN'],[PROPN],[PROPN],['O'],[],[B-MISC],""" do n't move on from the gospel , move into t...",""" do n't move on from the <[gospel]> , move in..."
...,...,...,...,...,...,...,...,...,...
966,@user70,['PROPN'],[PROPN],[PROPN],['O'],[],[B-PER],rt @user2041 : @user70 happy birthday man ! le...,rt @user2041 : <[@user70]> happy birthday man ...
967,@user660,['PROPN'],[X],[X],['O'],[],[B-PER],rt @user1157 : happy birthday @user660 ! url1209,rt @user1157 : happy birthday <[@user660]> ! u...
968,@user2046,['PROPN'],[X],[X],['O'],[],[B-PER],@user2046 further you do the swedish social se...,<[@user2046]> further you do the swedish socia...
969,@user2242,['PROPN'],[X],[X],['O'],[],[B-PER],@user2046 further you do the swedish social se...,@user2046 further you do the swedish social se...


In [7]:
new_df.to_csv('logs/TRAIN_relabelled_data_spacy_pass_entire_sentence_no_break_apart.tsv', sep='\t', index=False)